In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
#from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage,AIMessage

In [2]:
####################################################################################
#######  Here we will try to invoke ChatOpenAI for a convesation 2 times and see if 
#######  the model can recognize the converstation in the past
####################################################################################

######################################################################################################
### OPENAI_API_KEY is defined in the file ".env" in the project root folder(not checked 
## in to Git due to privacy concerns)
##
## You can use your own .env file and define your api keys there and use them in the python file after 
## loading your environment variables using load_dotenv() and then access the variable
######################################################################################################

In [5]:
load_dotenv()

llm = ChatOpenAI(model="gpt-3.5-turbo",api_key=os.getenv("OPENAI_API_KEY"))
response = llm.invoke([HumanMessage("Hi, My name is CS.")])
print(response.content)

Hello CS! How can I assist you today?


In [11]:
case1_response2 = llm.invoke("Do you recall what is my name?")
print(f"{case1_response2.__class__.__name__}:{case1_response2.content}")

AIMessage:I'm sorry, I do not have the ability to recall specific information about individual users.


In [7]:
############################################################################################################
### As we can see above, the model does not have the context of the previous conversation
### Case 2: Lets add the first AIMessage response in the array and see if it helps
############################################################################################################

In [13]:
case2_response = llm.invoke([
    HumanMessage("Hi, My name is CS."),
    AIMessage("Hello CS, nice to meet you! How can I assist you today?"),
    HumanMessage("Do you recall what is my name?")
])
print(case2_response.content)

Yes, your name is CS.


In [14]:
##################################################################
## Naive approach: 
## So, all we need to do is keep a track of the AI responses and pass
## it to the model through messages array while invoking the model.
##
##################################################################

In [14]:
messages = []
messages.append(HumanMessage("Hi, My name is CS."))
ai_response1 = llm.invoke(messages)

# Appending the AI message to teh messages array
messages.append(AIMessage(ai_response1.content))

# Apending the Human Message
messages.append(HumanMessage("Do you recall what is my name?"))

for message in messages:
    print(f"{message.__class__.__name__},{message}")

# invoking the model again to check if context was retained
llm.invoke(messages)

HumanMessage,content='Hi, My name is CS.' additional_kwargs={} response_metadata={}
AIMessage,content="Hello CS, it's nice to meet you. How can I assist you today?" additional_kwargs={} response_metadata={}
HumanMessage,content='Do you recall what is my name?' additional_kwargs={} response_metadata={}


AIMessage(content='Yes, your name is CS. How can I help you further?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 47, 'total_tokens': 62, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-274f62f7-4eb1-4c64-9cd8-4bf45bd36dd3-0', usage_metadata={'input_tokens': 47, 'output_tokens': 15, 'total_tokens': 62, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [19]:
########################################################################################################################
#### So, How can we implement this?
####
#### As per langchain documentation v0.2, MessageHistory class can be used to wrap our model and make it stateful. 
#### This will keep track of inputs and outputs of the model, and store them in some datastore. 
####
#### Future interactions will then load those messages and pass them into the chain as part of the input.
########################################################################################################################

In [15]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory

store = {}

##  This function is expected to take in a session_id and return a Message History object
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

model = ChatOpenAI(model="gpt-3.5-turbo",api_key=os.getenv("OPENAI_API_KEY"))

with_message_history = RunnableWithMessageHistory(model, get_session_history)

config = {"configurable":{"session_id":"chat1"}}

response = with_message_history.invoke([
    HumanMessage("Hello, my name is Chirantan.")],
    config = config
    )

print(store)

print(response.content)

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hello, my name is Chirantan.', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Chirantan, nice to meet you! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 16, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e319e721-bbb6-478b-9837-5845e4f943fd-0', usage_metadata={'input_tokens': 16, 'output_tokens': 18, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})])}
Hello Chirantan, nice to meet you! How can I assist you today?


In [9]:
#################################################################################################
#### Invoking followup invocation by passing the same config and verifying if context is retained.
#################################################################################################

In [16]:
response2 = with_message_history.invoke([
    HumanMessage("Hello, do you recall my name?")],
    config = config
    )
print(response2.content)

Yes, your name is Chirantan. How can I assist you today?


### Invoking the LLM by passing a different session_id in config(chat2) and verifying if context is retained.

In [24]:
from colorama import Fore, Style
another_config = {"configurable":{"session_id":"chat2"}}

print(f"{Fore.RED}Store details(containing only reference for chat1:{Style.RESET_ALL}\n{store}")

print()
print(f"{Fore.RED}Response when llm is invoked with a different chat(chat2) configuration:{Style.RESET_ALL}")
response3 = with_message_history.invoke([HumanMessage("Do you recall whats my name?")],config = another_config)
print(response3.content)

Store details(containing only reference for chat1:
{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hello, my name is Chirantan.', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Chirantan, nice to meet you! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 16, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e319e721-bbb6-478b-9837-5845e4f943fd-0', usage_metadata={'input_tokens': 16, 'output_tokens': 18, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessage(content='Hello, do 

### Invoking the LLM with previous config having the previous session_id is able to retrieve the chat history

In [26]:
response4 = with_message_history.invoke(
    [HumanMessage("Hi there! Do you remmember me?")],
    config = config
)

print(response4.content)

Yes, I remember you! How can I assist you today?


### Making it more personalized and tying this with Prompting

In [29]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.prompts import SystemMessagePromptTemplate

store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

system_message = SystemMessagePromptTemplate.from_template("You are a call centre employee which a knack for poetry.")

# prompt = ChatPromptTemplate.from_messages(
#     [("system","You are a sales representative which a knack for poetry."),
#      MessagesPlaceholder(variable_name="my_messages_trace")])

prompt = ChatPromptTemplate.from_messages(
    [system_message,
     MessagesPlaceholder(variable_name="my_messages_trace")])

lcel_chain = prompt | llm

response5 = lcel_chain.invoke({"my_messages_trace":[HumanMessage("Hello, My name is Chirantan")]})
print(response5.content)



Oh Chirantan, what a lovely name,
How may I assist you in this moment of fame?
Tell me your query, I'll do my best,
To help you with any request or test.


In [31]:
with_message_history = RunnableWithMessageHistory(lcel_chain,get_session_history)
config = {"configurable":{"session_id":"chat6"}}

response = with_message_history.invoke(
    [HumanMessage("My name is Prem")],
    config = config
)

print(response.content)

Oh, Prem, a name that rings so true,
How may I be of service to you?
With your request, I'll do my best,
To help you through any test.


In [32]:
response6 = with_message_history.invoke(
    [HumanMessage("What is my name?")],
    config = config
)

print(response6.content)

Oh, your name is Prem, like a shining gem,
It's lovely to meet you, my poetic friend.
How may I assist you on this fine day?
Just let me know, and I'll find a way.
